In [7]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


### Neighborhood has a total of 5 boroughs and 306 neighborhoods.
In order to segement the neighborhoods and explore them, we will  need a dataset that contains the 5 boroughs and the neighborhoods with latitude and logitude coordinates of each neighborhood. 


In [8]:
!wget -q -O 'newyork_data.json' https://ibm.box.com/shared/static/fbpwbovar7lf8p5sgddm06cgipa2rxpe.json
print('Data downloaded!')

Data downloaded!


In [9]:
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)


In [10]:
newyork_data

{'bbox': [-74.2492599487305,
  40.5033187866211,
  -73.7061614990234,
  40.9105606079102],
 'crs': {'properties': {'name': 'urn:ogc:def:crs:EPSG::4326'}, 'type': 'name'},
 'features': [{'geometry': {'coordinates': [-73.84720052054902,
     40.89470517661],
    'type': 'Point'},
   'geometry_name': 'geom',
   'id': 'nyu_2451_34572.1',
   'properties': {'annoangle': 0.0,
    'annoline1': 'Wakefield',
    'annoline2': None,
    'annoline3': None,
    'bbox': [-73.84720052054902,
     40.89470517661,
     -73.84720052054902,
     40.89470517661],
    'borough': 'Bronx',
    'name': 'Wakefield',
    'stacked': 1},
   'type': 'Feature'},
  {'geometry': {'coordinates': [-73.82993910812398, 40.87429419303012],
    'type': 'Point'},
   'geometry_name': 'geom',
   'id': 'nyu_2451_34572.2',
   'properties': {'annoangle': 0.0,
    'annoline1': 'Co-op',
    'annoline2': 'City',
    'annoline3': None,
    'bbox': [-73.82993910812398,
     40.87429419303012,
     -73.82993910812398,
     40.874294193

In [11]:
neighborhoods_data = newyork_data['features']

In [12]:
neighborhoods_data[0]

{'geometry': {'coordinates': [-73.84720052054902, 40.89470517661],
  'type': 'Point'},
 'geometry_name': 'geom',
 'id': 'nyu_2451_34572.1',
 'properties': {'annoangle': 0.0,
  'annoline1': 'Wakefield',
  'annoline2': None,
  'annoline3': None,
  'bbox': [-73.84720052054902,
   40.89470517661,
   -73.84720052054902,
   40.89470517661],
  'borough': 'Bronx',
  'name': 'Wakefield',
  'stacked': 1},
 'type': 'Feature'}

In [13]:
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

In [14]:
neighborhoods = pd.DataFrame(columns=column_names)


In [15]:
for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

In [16]:
neighborhoods.head()


,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


In [17]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(neighborhoods['Borough'].unique()),
        neighborhoods.shape[0]
    )
)

The dataframe has 5 boroughs and 306 neighborhoods.


In [18]:
address = 'New York City, NY'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))


/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/ipykernel/__main__.py:3: DeprecationWarning: Using Nominatim with the default "geopy/1.19.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  app.launch_new_instance()


The geograpical coordinate of New York City are 40.7127281, -74.0060152.


In [19]:
# create map of New York using latitude and longitude values
map_newyork = folium.Map(location=[latitude, longitude], zoom_start=10)

In [20]:
# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=1,
        parse_html=False).add_to(map_newyork)  
    
map_newyork


In [21]:
manhattan_data = neighborhoods[neighborhoods['Borough'] == 'Manhattan'].reset_index(drop=True)
manhattan_data.head()

,Borough,Neighborhood,Latitude,Longitude
0,Manhattan,Marble Hill,40.876551,-73.910660
1,Manhattan,Chinatown,40.715618,-73.994279
2,Manhattan,Washington Heights,40.851903,-73.936900
3,Manhattan,Inwood,40.867684,-73.921210
4,Manhattan,Hamilton Heights,40.823604,-73.949688


In [22]:

address = 'Manhattan, NY'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Manhattan are {}, {}.'.format(latitude, longitude))


/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/ipykernel/__main__.py:4: DeprecationWarning: Using Nominatim with the default "geopy/1.19.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.


The geograpical coordinate of Manhattan are 40.7900869, -73.9598295.


In [23]:
# create map of Manhattan using latitude and longitude values
map_manhattan = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(manhattan_data['Latitude'], manhattan_data['Longitude'], manhattan_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_manhattan)  
    
map_manhattan

In [25]:
CLIENT_ID = 'L0YKBUZGFO0M1O4XWQW5L4GXTBU01SKSXFRGCRVMCAKTLTWL' # your Foursquare ID
CLIENT_SECRET = 'GCGBDKGZGFPZCR1KNGC1N2EBPUQXHYNYIYNBTASKRVV4YAUX' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

In [26]:
manhattan_data.loc[0, 'Neighborhood']


'Marble Hill'

In [27]:
neighborhood_latitude = manhattan_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = manhattan_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = manhattan_data.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))



Latitude and longitude values of Marble Hill are 40.87655077879964, -73.91065965862981.


In [28]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius
# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL


'https://api.foursquare.com/v2/venues/explore?&client_id=L0YKBUZGFO0M1O4XWQW5L4GXTBU01SKSXFRGCRVMCAKTLTWL&client_secret=GCGBDKGZGFPZCR1KNGC1N2EBPUQXHYNYIYNBTASKRVV4YAUX&v=20180605&ll=40.87655077879964,-73.91065965862981&radius=500&limit=100'

In [29]:
results = requests.get(url).json()
results


{'meta': {'code': 200, 'requestId': '5cc610cb1ed2196c3ffc5adf'},
 'response': {'groups': [{'items': [{'reasons': {'count': 0,
       'items': [{'reasonName': 'globalInteractionReason',
         'summary': 'This spot is popular',
         'type': 'general'}]},
      'referralId': 'e-0-4b4429abf964a52037f225e3-0',
      'venue': {'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/pizza_',
          'suffix': '.png'},
         'id': '4bf58dd8d48988d1ca941735',
         'name': 'Pizza Place',
         'pluralName': 'Pizza Places',
         'primary': True,
         'shortName': 'Pizza'}],
       'delivery': {'id': '72548',
        'provider': {'icon': {'name': '/delivery_provider_seamless_20180129.png',
          'prefix': 'https://fastly.4sqi.net/img/general/cap/',
          'sizes': [40, 50]},
         'name': 'seamless'},
        'url': 'https://www.seamless.com/menu/arturos-pizza-5189-broadway-ave-new-york/72548?affiliate=1131&utm_source=foursquare-affiliat

In [30]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']


In [33]:
venues = results['response']['groups'][0]['items']
#     
nearby_venues = json_normalize(venues) # flatten JSON
# 
# # filter columns
 filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
   nearby_venues.loc[:, filtered_columns]
# 
# # filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)
# 
# # clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]
# 
nearby_venues.head()


,count,items,referralId,categories,id,name,prefix,sizes,name,url,id,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,name,count,groups,id
0,0,"[{'reasonName': 'globalInteractionReason', 'ty...",e-0-4b4429abf964a52037f225e3-0,Pizza Place,72548,/delivery_provider_seamless_20180129.png,https://fastly.4sqi.net/img/general/cap/,"[40, 50]",seamless,https://www.seamless.com/menu/arturos-pizza-51...,4b4429abf964a52037f225e3,5198 Broadway,US,New York,United States,at 225th St.,240,"[5198 Broadway (at 225th St.), New York, NY 10...","[{'lat': 40.87441177110231, 'lng': -73.9102710...",40.874412,-73.910271,NaN,10463,NY,Arturo's,0,[],NaN
1,0,"[{'reasonName': 'globalInteractionReason', 'ty...",e-0-4baf59e8f964a520a6f93be3-1,Yoga Studio,NaN,NaN,NaN,NaN,NaN,NaN,4baf59e8f964a520a6f93be3,5500 Broadway,US,Bronx,United States,230th Street,376,"[5500 Broadway (230th Street), Bronx, NY 10463...","[{'lat': 40.876843690797934, 'lng': -73.906203...",40.876844,-73.906204,NaN,10463,NY,Bikram Yoga,0,[],NaN
2,0,"[{'reasonName': 'globalInteractionReason', 'ty...",e-0-4b79cc46f964a520c5122fe3-2,Diner,NaN,NaN,NaN,NaN,NaN,NaN,4b79cc46f964a520c5122fe3,3033 Tibbett Ave,US,Bronx,United States,btwn 230th & 231st,452,"[3033 Tibbett Ave (btwn 230th & 231st), Bronx,...","[{'lat': 40.8804044222466, 'lng': -73.90893738...",40.880404,-73.908937,NaN,10463,NY,Tibbett Diner,0,[],NaN
3,0,"[{'reasonName': 'globalInteractionReason', 'ty...",e-0-55f81cd2498ee903149fcc64-3,Coffee Shop,NaN,NaN,NaN,NaN,NaN,NaN,55f81cd2498ee903149fcc64,171 W 230th St,US,Bronx,United States,Kimberly Pl,441,"[171 W 230th St (Kimberly Pl), Bronx, NY 10463...","[{'lat': 40.87753134921497, 'lng': -73.9055821...",40.877531,-73.905582,NaN,10463,NY,Starbucks,0,[],NaN
4,0,"[{'reasonName': 'globalInteractionReason', 'ty...",e-0-4b5357adf964a520319827e3-4,Donut Shop,NaN,NaN,NaN,NaN,NaN,NaN,4b5357adf964a520319827e3,5501 Broadway,US,Bronx,United States,W 230th St,342,"[5501 Broadway (W 230th St), Bronx, NY 10463, ...","[{'lat': 40.87713584201589, 'lng': -73.9066655...",40.877136,-73.906666,NaN,10463,NY,Dunkin',0,[],NaN


In [62]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

25 venues were returned by Foursquare.


#### Explore neighborhood in Manhattan

In [76]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [77]:
manhattan_venues = getNearbyVenues(names=manhattan_data['Neighborhood'],
                                   latitudes=manhattan_data['Latitude'],
                                   longitudes=manhattan_data['Longitude']
                                  )

Marble Hill
Chinatown
Washington Heights
Inwood
Hamilton Heights
Manhattanville
Central Harlem
East Harlem
Upper East Side
Yorkville
Lenox Hill
Roosevelt Island
Upper West Side
Lincoln Square
Clinton
Midtown
Murray Hill
Chelsea
Greenwich Village
East Village
Lower East Side
Tribeca
Little Italy
Soho
West Village
Manhattan Valley
Morningside Heights
Gramercy
Battery Park City
Financial District
Carnegie Hill
Noho
Civic Center
Midtown South
Sutton Place
Turtle Bay
Tudor City
Stuyvesant Town
Flatiron
Hudson Yards


In [78]:
print (manhattan_venues.shape)
manhattan_venues.head()


(3322, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Marble Hill,40.876551,-73.91066,Arturo's,40.874412,-73.910271,Pizza Place
1,Marble Hill,40.876551,-73.91066,Bikram Yoga,40.876844,-73.906204,Yoga Studio
2,Marble Hill,40.876551,-73.91066,Tibbett Diner,40.880404,-73.908937,Diner
3,Marble Hill,40.876551,-73.91066,Starbucks,40.877531,-73.905582,Coffee Shop
4,Marble Hill,40.876551,-73.91066,Dunkin',40.877136,-73.906666,Donut Shop


In [79]:
manhattan_venues.groupby('Neighborhood').count()


,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Battery Park City,100,100,100,100,100,100
Carnegie Hill,100,100,100,100,100,100
Central Harlem,44,44,44,44,44,44
Chelsea,100,100,100,100,100,100
Chinatown,100,100,100,100,100,100
Civic Center,100,100,100,100,100,100
Clinton,100,100,100,100,100,100
East Harlem,41,41,41,41,41,41
East Village,100,100,100,100,100,100


In [80]:
print('There are {} uniques categories.'.format(len(manhattan_venues['Venue Category'].unique())))


There are 333 uniques categories.


#### Analyse each neighborhood

In [81]:
# one hot encoding
manhattan_onehot = pd.get_dummies(manhattan_venues[['Venue Category']], prefix="", prefix_sep="")


In [82]:
# add neighborhood column back to dataframe
manhattan_onehot['Neighborhood'] = manhattan_venues['Neighborhood'] 

In [83]:
# move neighborhood column to the first column
fixed_columns = [manhattan_onehot.columns[-1]] + list(manhattan_onehot.columns[:-1])
manhattan_onehot = manhattan_onehot[fixed_columns]

In [84]:
manhattan_onehot.head()


,Neighborhood,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,American Restaurant,Antique Shop,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auditorium,Australian Restaurant,Austrian Restaurant,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Cambodian Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Caucasian Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Circus,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Bookstore,College Cafeteria,College Gym,College Theater,Comedy Club,Community Center,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Cuban Restaurant,Cultural Center,Cupcake Shop,Cycle Studio,Czech Restaurant,Dance Studio,Daycare,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Donut Shop,Drugstore,Dry Cleaner,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General College & University,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,Herbs & Spices Store,High School,Himalayan Restaurant,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Korean Restaurant,Kosher Restaurant,Latin American Restaurant,Laundry Service,Leather Goods Store,Lebanese Restaurant,Library,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music School,Music Store,Music Venue,Nail Salon,New American Restaurant,Newsstand,Nightclub,Non-Profit,Noodle House,North Indian Restaurant,Office,Opera House,Optical Shop,Organic Grocery,Other Nightlife,Outdoor Sculpture,Outdoors & Recreation,Paella Restaurant,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Peruvian Restaurant,Pet Café,Pet Service,Pet Store,Pharmacy,Photography Studio,Piano Bar,Pie Shop,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Pub,Public Art,Ramen Restaurant,Record Shop,Rental Car Location,Residential Building (Apartment / Condo),Resort,Rest Area,Restaurant,River,Rock Climbing Spot,Rock Club,Roof Deck,Russian Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shanghai Restaurant,Shipping Store,Shoe Repair,Shoe Store,Shopping Mall,Skate Park,Ski Shop,Smoke Shop,Snack Place

In [85]:
manhattan_onehot.shape

(3322, 334)

In [86]:
manhattan_grouped = manhattan_onehot.groupby('Neighborhood').mean().reset_index()
manhattan_grouped

,Neighborhood,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,American Restaurant,Antique Shop,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auditorium,Australian Restaurant,Austrian Restaurant,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Cambodian Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Caucasian Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Circus,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Bookstore,College Cafeteria,College Gym,College Theater,Comedy Club,Community Center,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Cuban Restaurant,Cultural Center,Cupcake Shop,Cycle Studio,Czech Restaurant,Dance Studio,Daycare,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Donut Shop,Drugstore,Dry Cleaner,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General College & University,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,Herbs & Spices Store,High School,Himalayan Restaurant,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Korean Restaurant,Kosher Restaurant,Latin American Restaurant,Laundry Service,Leather Goods Store,Lebanese Restaurant,Library,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music School,Music Store,Music Venue,Nail Salon,New American Restaurant,Newsstand,Nightclub,Non-Profit,Noodle House,North Indian Restaurant,Office,Opera House,Optical Shop,Organic Grocery,Other Nightlife,Outdoor Sculpture,Outdoors & Recreation,Paella Restaurant,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Peruvian Restaurant,Pet Café,Pet Service,Pet Store,Pharmacy,Photography Studio,Piano Bar,Pie Shop,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Pub,Public Art,Ramen Restaurant,Record Shop,Rental Car Location,Residential Building (Apartment / Condo),Resort,Rest Area,Restaurant,River,Rock Climbing Spot,Rock Club,Roof Deck,Russian Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shanghai Restaurant,Shipping Store,Shoe Repair,Shoe Store,Shopping Mall,Skate Park,Ski Shop,Smoke Shop,Snack Place

In [87]:
num_top_venues = 5

for hood in manhattan_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = manhattan_grouped[manhattan_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')


----Battery Park City----
           venue  freq
0    Coffee Shop  0.07
1           Park  0.07
2          Hotel  0.05
3            Gym  0.04
4  Shopping Mall  0.03


----Carnegie Hill----
         venue  freq
0  Pizza Place  0.06
1  Coffee Shop  0.05
2         Café  0.04
3          Gym  0.03
4          Spa  0.03


----Central Harlem----
                  venue  freq
0    Chinese Restaurant  0.07
1    African Restaurant  0.07
2     French Restaurant  0.05
3  Gym / Fitness Center  0.05
4    Seafood Restaurant  0.05


----Chelsea----
                venue  freq
0         Coffee Shop  0.07
1      Ice Cream Shop  0.05
2  Italian Restaurant  0.05
3           Nightclub  0.04
4              Bakery  0.04


----Chinatown----
                   venue  freq
0     Chinese Restaurant  0.10
1     Dim Sum Restaurant  0.04
2  Vietnamese Restaurant  0.04
3           Cocktail Bar  0.04
4        Bubble Tea Shop  0.04


----Civic Center----
                  venue  freq
0    Italian Restaurant  0.05
1     

In [88]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [89]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))


In [90]:
# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = manhattan_grouped['Neighborhood']

for ind in np.arange(manhattan_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(manhattan_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Battery Park City,Park,Coffee Shop,Hotel,Gym,Italian Restaurant,Clothing Store,Shopping Mall,Wine Shop,Cupcake Shop,Burger Joint
1,Carnegie Hill,Pizza Place,Coffee Shop,Café,Gym,Cosmetics Shop,Bar,Spa,French Restaurant,Bookstore,Grocery Store
2,Central Harlem,Chinese Restaurant,African Restaurant,French Restaurant,Cosmetics Shop,Gym / Fitness Center,Public Art,Seafood Restaurant,American Restaurant,Cafeteria,Event Space
3,Chelsea,Coffee Shop,Italian Restaurant,Ice Cream Shop,Bakery,Nightclub,Theater,American Restaurant,Seafood Restaurant,Hotel,Cupcake Shop
4,Chinatown,Chinese Restaurant,Dim Sum Restaurant,American Restaurant,Cocktail Bar,Vietnamese Restaurant,Bubble Tea Shop,Bakery,Dumpling Restaurant,Noodle House,Hotpot Restaurant
5,Civic Center,Italian Restaurant,Bakery,Gym / Fitness Center,French Restaurant,Coffee Shop,Yoga Studio,Spa,Sandwich Place,Cocktail Bar,Park
6,Clinton,Theater,Italian Restaurant,Gym / Fitness Center,American Restaurant,Hotel,Wine Shop,Spa,Cocktail Bar,New American Restaurant,Dog Run
7,East Harlem,Mexican Restaurant,Bakery,Latin American Restaurant,Deli / Bodega,Thai Restaurant,Spa,Sandwich Place,Gym,Grocery Store,Cocktail Bar
8,East Village,Bar,Wine Bar,Pizza Place,Ice Cream Shop,Mexican Restaurant,Vegetarian / Vegan Restaurant,Chinese Restaurant,Ramen Restaurant,Coffee Shop,Cocktail Bar
9,Financial District,Coffee Shop,Hotel,Wine Shop,Bar,Italian Restaurant,Gym,Steakhouse,Pizza Place,Food Truck,Park


#### Cluster Neighbor

In [91]:
# set number of clusters
kclusters = 5

manhattan_grouped_clustering = manhattan_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(manhattan_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 


array([0, 2, 2, 2, 2, 2, 1, 4, 2, 2], dtype=int32)

In [92]:
manhattan_merged = manhattan_data

# add clustering labels
manhattan_merged['Cluster Labels'] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
manhattan_merged = manhattan_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

manhattan_merged # check the last columns!

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Manhattan,Marble Hill,40.876551,-73.910660,0,Coffee Shop,Discount Store,Yoga Studio,Tennis Stadium,Big Box Store,Supplement Shop,Steakhouse,Shoe Store,Seafood Restaurant,Sandwich Place
1,Manhattan,Chinatown,40.715618,-73.994279,2,Chinese Restaurant,Dim Sum Restaurant,American Restaurant,Cocktail Bar,Vietnamese Restaurant,Bubble Tea Shop,Bakery,Dumpling Restaurant,Noodle House,Hotpot Restaurant
2,Manhattan,Washington Heights,40.851903,-73.936900,2,Café,Bakery,Mobile Phone Shop,Grocery Store,Shoe Store,Liquor Store,New American Restaurant,Latin American Restaurant,Pizza Place,Gym
3,Manhattan,Inwood,40.867684,-73.921210,2,Mexican Restaurant,Café,Pizza Place,Lounge,Frozen Yogurt Shop,Park,Restaurant,Bakery,Chinese Restaurant,Deli / Bodega
4,Manhattan,Hamilton Heights,40.823604,-73.949688,2,Mexican Restaurant,Café,Coffee Shop,Pizza Place,Yoga Studio,Sushi Restaurant,Caribbean Restaurant,Chinese Restaurant,School,Bakery
5,Manhattan,Manhattanville,40.816934,-73.957385,2,Coffee Shop,Italian Restaurant,Liquor Store,Mexican Restaurant,Seafood Restaurant,Park,Japanese Curry Restaurant,Ramen Restaurant,Sushi Restaurant,Supermarket
6,Manhattan,Central Harlem,40.815976,-73.943211,1,Chinese Restaurant,African Restaurant,French Restaurant,Cosmetics Shop,Gym / Fitness Center,Public Art,Seafood Restaurant,American Restaurant,Cafeteria,Event Space
7,Manhattan,East Harlem,40.792249,-73.944182,4,Mexican Restaurant,Bakery,Latin American Restaurant,Deli / Bodega,Thai Restaurant,Spa,Sandwich Place,Gym,Grocery Store,Cocktail Bar
8,Manhattan,Upper East Side,40.775639,-73.960508,2,Italian Restaurant,Coffee Shop,Exhibit,Bakery,Gym / Fitness Center,Art Gallery,Cocktail Bar,French Restaurant,Juice Bar,Spa
9,Manhattan,Yorkville,40.775930,-73.947118,2,Italian Restaurant,Coffee Shop,Bar,Gym,Pizza Place,Deli / Bodega,Wine Shop,Diner,Japanese Restaurant,Mexican Restaurant


In [93]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(manhattan_merged['Latitude'], manhattan_merged['Longitude'], manhattan_merged['Neighborhood'], manhattan_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

#### Cluster1

In [94]:
cluster1 = manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 0, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]
cluster1

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Marble Hill,Coffee Shop,Discount Store,Yoga Studio,Tennis Stadium,Big Box Store,Supplement Shop,Steakhouse,Shoe Store,Seafood Restaurant,Sandwich Place
21,Tribeca,Park,Italian Restaurant,Spa,Boutique,Café,American Restaurant,Coffee Shop,Wine Bar,Wine Shop,Gym
25,Manhattan Valley,Indian Restaurant,Coffee Shop,Pizza Place,Yoga Studio,Bar,Café,Deli / Bodega,Mexican Restaurant,Playground,Thai Restaurant
28,Battery Park City,Park,Coffee Shop,Hotel,Gym,Italian Restaurant,Clothing Store,Shopping Mall,Wine Shop,Cupcake Shop,Burger Joint


#### Cluster 2

In [95]:
cluster2 = manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 1, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]
cluster2

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,Central Harlem,Chinese Restaurant,African Restaurant,French Restaurant,Cosmetics Shop,Gym / Fitness Center,Public Art,Seafood Restaurant,American Restaurant,Cafeteria,Event Space
17,Chelsea,Coffee Shop,Italian Restaurant,Ice Cream Shop,Bakery,Nightclub,Theater,American Restaurant,Seafood Restaurant,Hotel,Cupcake Shop


#### Cluster 3

In [96]:
cluster3 = manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 2, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]
cluster3

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Chinatown,Chinese Restaurant,Dim Sum Restaurant,American Restaurant,Cocktail Bar,Vietnamese Restaurant,Bubble Tea Shop,Bakery,Dumpling Restaurant,Noodle House,Hotpot Restaurant
2,Washington Heights,Café,Bakery,Mobile Phone Shop,Grocery Store,Shoe Store,Liquor Store,New American Restaurant,Latin American Restaurant,Pizza Place,Gym
3,Inwood,Mexican Restaurant,Café,Pizza Place,Lounge,Frozen Yogurt Shop,Park,Restaurant,Bakery,Chinese Restaurant,Deli / Bodega
4,Hamilton Heights,Mexican Restaurant,Café,Coffee Shop,Pizza Place,Yoga Studio,Sushi Restaurant,Caribbean Restaurant,Chinese Restaurant,School,Bakery
5,Manhattanville,Coffee Shop,Italian Restaurant,Liquor Store,Mexican Restaurant,Seafood Restaurant,Park,Japanese Curry Restaurant,Ramen Restaurant,Sushi Restaurant,Supermarket
8,Upper East Side,Italian Restaurant,Coffee Shop,Exhibit,Bakery,Gym / Fitness Center,Art Gallery,Cocktail Bar,French Restaurant,Juice Bar,Spa
9,Yorkville,Italian Restaurant,Coffee Shop,Bar,Gym,Pizza Place,Deli / Bodega,Wine Shop,Diner,Japanese Restaurant,Mexican Restaurant
10,Lenox Hill,Italian Restaurant,Coffee Shop,Sushi Restaurant,Pizza Place,Gym / Fitness Center,Cosmetics Shop,Sporting Goods Shop,Deli / Bodega,Gym,Burger Joint
11,Roosevelt Island,Sandwich Place,Park,Gym,Kosher Restaurant,Supermarket,Farmers Market,Metro Station,Outdoors & Recreation,Greek Restaurant,School
12,Upper West Side,Italian Restaurant,Wine Bar,Bar,Indian Restaurant,Coffee Shop,Bakery,Mediterranean Restaurant,Vegetarian / Vegan Restaurant,Thai Restaurant,Seafood Restaurant


In [ ]:
# #### Cluster 4

In [97]:
cluster4 = manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 3, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]
cluster4

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
30,Carnegie Hill,Pizza Place,Coffee Shop,Café,Gym,Cosmetics Shop,Bar,Spa,French Restaurant,Bookstore,Grocery Store


In [ ]:
# #### Cluster 5


In [98]:
cluster5 = manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 4, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]
cluster5

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
7,East Harlem,Mexican Restaurant,Bakery,Latin American Restaurant,Deli / Bodega,Thai Restaurant,Spa,Sandwich Place,Gym,Grocery Store,Cocktail Bar
13,Lincoln Square,Theater,Gym / Fitness Center,Concert Hall,Café,Plaza,Italian Restaurant,Performing Arts Venue,French Restaurant,Opera House,Park
15,Midtown,Hotel,Clothing Store,Cocktail Bar,Theater,Steakhouse,Spa,Bakery,Coffee Shop,Bookstore,Food Truck
19,East Village,Bar,Wine Bar,Pizza Place,Ice Cream Shop,Mexican Restaurant,Vegetarian / Vegan Restaurant,Chinese Restaurant,Ramen Restaurant,Coffee Shop,Cocktail Bar
20,Lower East Side,Café,Ramen Restaurant,Coffee Shop,Art Gallery,Pizza Place,Cocktail Bar,Chinese Restaurant,Shoe Store,Bakery,Sandwich Place
22,Little Italy,Bakery,Café,Salon / Barbershop,Sandwich Place,Seafood Restaurant,Clothing Store,Ice Cream Shop,Bubble Tea Shop,Mediterranean Restaurant,Chinese Restaurant
33,Midtown South,Korean Restaurant,Hotel,Hotel Bar,Japanese Restaurant,Coffee Shop,Bakery,Italian Restaurant,Cocktail Bar,Boutique,Cosmetics Shop
37,Stuyvesant Town,Bar,Park,Playground,Boat or Ferry,Basketball Court,Baseball Field,Cocktail Bar,Coffee Shop,Heliport,Farmers Market


In [99]:
clusters=pd.DataFrame({"Cluster 1":cluster1["Neighborhood"],
                      "Cluster 2":cluster2["Neighborhood"],
                      "Cluster 3":cluster3["Neighborhood"],
                      "Cluster 4":cluster4["Neighborhood"],
                      "Cluster 5":cluster5["Neighborhood"]
                       })

In [100]:
clusters = clusters.replace(np.nan, '', regex=True)

In [101]:
clusters

,Cluster 1,Cluster 2,Cluster 3,Cluster 4,Cluster 5
0,Marble Hill,,,,
1,,,Chinatown,,
2,,,Washington Heights,,
3,,,Inwood,,
4,,,Hamilton Heights,,
5,,,Manhattanville,,
6,,Central Harlem,,,
7,,,,,East Harlem
8,,,Upper East Side,,
9,,,Yorkville,,


In [103]:
print ("Neighborhood has more vistors to Coffee shop are\n")
manhattan_merged[manhattan_merged['1st Most Common Venue'] == 'Coffee Shop']


Neighborhood has more vistors to Coffee shop are



,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Manhattan,Marble Hill,40.876551,-73.910660,0,Coffee Shop,Discount Store,Yoga Studio,Tennis Stadium,Big Box Store,Supplement Shop,Steakhouse,Shoe Store,Seafood Restaurant,Sandwich Place
5,Manhattan,Manhattanville,40.816934,-73.957385,2,Coffee Shop,Italian Restaurant,Liquor Store,Mexican Restaurant,Seafood Restaurant,Park,Japanese Curry Restaurant,Ramen Restaurant,Sushi Restaurant,Supermarket
17,Manhattan,Chelsea,40.744035,-74.003116,1,Coffee Shop,Italian Restaurant,Ice Cream Shop,Bakery,Nightclub,Theater,American Restaurant,Seafood Restaurant,Hotel,Cupcake Shop
26,Manhattan,Morningside Heights,40.808000,-73.963896,2,Coffee Shop,Bookstore,American Restaurant,Park,Tennis Court,Deli / Bodega,Burger Joint,Food Truck,Café,Indian Restaurant
29,Manhattan,Financial District,40.707107,-74.010665,2,Coffee Shop,Hotel,Wine Shop,Bar,Italian Restaurant,Gym,Steakhouse,Pizza Place,Food Truck,Park


## Conclusion :

This analysis concludes that 9 neighborhoods having most frequent vistors to coffeeshop and are preferable location to open a new Angel Coffee Shop
